In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    tables = soup.find_all("table", {"class": "wikitable"})

    launch_table = tables[3] 

    headers = [header.text.strip() for header in launch_table.find_all("th")]

    rows = []
    for row in launch_table.find_all("tr"):
        cells = row.find_all(["th", "td"])
        row_data = [cell.text.strip().replace("\n", " ") for cell in cells]

        if len(row_data) == len(headers):
            rows.append(row_data)

    df = pd.DataFrame(rows, columns=headers)

    df = df[~df["Date and time (UTC)"].str.contains("Date", na=False)]  

    df.to_csv("spacex_launches_scraped.csv", index=False)

    print("✅ Web Scraping Successful! Data saved to 'spacex_launches_scraped.csv'")
    print(df.head())

else:
    print(f"❌ Error: Unable to fetch data. HTTP Status Code: {response.status_code}")


✅ Web Scraping Successful! Data saved to 'spacex_launches_scraped.csv'
  Date and time (UTC) Version,booster[f]             Launch site  \
1  18 March 202518:09              F9 B5  Cape Canaveral, SLC‑40   
2  20 March 202506:02              F9 B5      Vandenberg, SLC‑4E   
3  22 March 202520:45              F9 B5      Vandenberg, SLC‑4E   
4          March 2025              F9 B5                TBA (FL)   
5          March 2025       F9 B5B1085.6                TBA (FL)   

                                          Payload[g]      Orbit    Customer  
1                              Starlink: Group 12-25        LEO      SpaceX  
2  NROL-57 (unknown number of Starshield satellites)        LEO         NRO  
3                               Starlink: Group 11-7        LEO      SpaceX  
4              Fram2 (Crew Dragon C207.4 Resilience)  Polar LEO   Chun Wang  
5                                         CRS SpX-32  LEO (ISS)  NASA (CRS)  
